In [1]:
import json
import pandas as pd
import logging
import re
import json
import numpy as np
import torch
import pickle
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from transformers import BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix

#train_data= pickle.load(open('train_data_700.pkl','rb')) #300 rows

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
import pandas as pd #701 rows
data = pd.read_json("/Users/cosmos/Desktop/Deepblue/ref - kaggle/pre_trained.json", lines = True)
#data = pd.read_json("/Users/cosmos/Desktop/Deepblue/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/NER_RESUME.json", lines = True)
data.columns

Index(['content', 'annotation', 'extras', 'metadata'], dtype='object')

In [3]:
#extra has no unique values we can drop it
#metadata can be dropped
data['annotation'][0]

[{'label': ['Email Address'],
  'points': [{'start': 1155,
    'end': 1198,
    'text': 'indeed.com/r/Afreen-Jamadar/8baf379b705e37c6'}]},
 {'label': ['Links'],
  'points': [{'start': 1143,
    'end': 1239,
    'text': 'https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN'}]},
 {'label': ['Skills'],
  'points': [{'start': 743,
    'end': 1140,
    'text': 'Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming Languages: C, C++, Java, .net, php.\n• Web Designing: HTML, XML\n• Operating Systems: Windows […] Windows Server 2003, Linux.\n• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 729, 'end': 732, 'text': '2016'}]},
 {'label': ['College Name'],
  'points': [{'start': 675,
    'end': 702,
    'text': 'Shiva

In [4]:
data["content"] = data["content"].apply(lambda x: x.replace("\n", " "))
data["content"].head(5)

0    Afreen Jamadar Active member of IIIT Committee...
1    Alok Khandai Operational Analyst (SQL DBA) Eng...
2    Anvitha Rao Automation developer  - Email me o...
3    arjun ks Senior Program coordinator - oracle I...
4    Arun Elumalai QA Tester  Chennai, Tamil Nadu -...
Name: content, dtype: object

In [5]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [6]:
data

,content,annotation,extras,metadata
0,Afreen Jamadar Active member of IIIT Committee...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN,"{'first_done_at': 1527844872000, 'last_updated..."
1,Alok Khandai Operational Analyst (SQL DBA) Eng...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN,"{'first_done_at': 1527845028000, 'last_updated..."
2,Anvitha Rao Automation developer - Email me o...,"[{'label': ['Links'], 'points': [{'start': 288...",NaN,"{'first_done_at': 1527744637000, 'last_updated..."
3,arjun ks Senior Program coordinator - oracle I...,"[{'label': ['Skills'], 'points': [{'start': 50...",NaN,"{'first_done_at': 1527834843000, 'last_updated..."
4,"Arun Elumalai QA Tester Chennai, Tamil Nadu -...","[{'label': ['Skills'], 'points': [{'start': 19...",NaN,"{'first_done_at': 1527847268000, 'last_updated..."
...,...,...,...,...
696,Prashant Duble Territory Sales Manager - Bhart...,"[{'label': ['College'], 'points': [{'start': 5...",NaN,"{'first_done_at': 1540021594000, 'last_updated..."
697,Rayees Parwez Store Manager - Sales & Operatio...,"[{'label': ['Skills'], 'points': [{'start': 48...",NaN,"{'first_done_at': 1539697381000, 'last_updated..."
698,Sachin Kushwah Territory Sales Manager - Essar...,"[{'label': ['Skills'], 'points': [{'start': 37...",NaN,"{'first_done_at': 1540021512000, 'last_updated..."
699,Sakshi Sundriyal Manager- Sales - Benefits and...,"[{'label': ['Links'], 'points': [{'start': 627...",NaN,"{'first_done_at': 1539697642000, 'last_updated..."


In [7]:
#data = trim_entity_spans(convert_dataturks_to_spacy("//Users/cosmos/Desktop/Deepblue/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/NER_RESUME-300.json"))
#data = trim_entity_spans(convert_dataturks_to_spacy("/Users/cosmos/Desktop/Deepblue/ref - kaggle/pre_trained.json"))
#data

In [8]:
# def clean_entities(training_data):
    
#     clean_data = []
#     for text, annotation in training_data:
        
#         entities = annotation.get('entities')
#         entities_copy = entities.copy()
        
#         # append entity only if it is longer than its overlapping entity
#         i = 0
#         for entity in entities_copy:
#             j = 0
#             for overlapping_entity in entities_copy:
#                 # Skip self
#                 if i != j:
#                     e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
#                     # Delete any entity that overlaps, keep if longer
#                     if ((e_start >= oe_start and e_start <= oe_end) \
#                     or (e_end <= oe_end and e_end >= oe_start)) \
#                     and ((e_end - e_start) <= (oe_end - oe_start)):
#                         entities.remove(entity)
#                 j += 1
#             i += 1
#         clean_data.append((text, {'entities': entities}))
                
#     return clean_data

# data = clean_entities(data)

In [9]:
# # Changing data to appropriate format so as to feed it to the model
# import nltk
# #nltk.download('stopwords')

# from nltk.corpus import stopwords
# en_stops = set(stopwords.words('english'))

# df_data = pd.DataFrame(columns = ["clean_content", "entities_mapped"])

# entities_mapped = []
# clean_content = []

# for i in range(len(data)):
#     content = data[i][0].split()
#     entities = data[i][1]["entities"]
#     words = []
#     labels = []
    
#     for word in content:
#         if (word.isalnum() or word.find(".com") != -1) and word not in en_stops:
#             words.append(word)
#             found = False
#             for entity in sorted(entities):
#                 ent_start = entity[0]
#                 ent_end = entity[1]
#                 ent_label = entity[2]

#                 if word in data[i][0][ent_start: ent_end].split(): 
#                     labels.append(ent_label)
#                     found = True
#                     break
#             if not found:
#                 labels.append("O")
    
#     entities_mapped.append(labels)
#     clean_content.append(words)
    
    
# df_data["entities_mapped"] = entities_mapped
# df_data["clean_content"] = clean_content
# df_data["clean_content"] = df_data["clean_content"].apply(lambda x: " ".join(x))

In [10]:
MAX_LEN = 500
EPOCHS = 6
# NUM_LABELS = 12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = '/Users/cosmos/Documents/BERT/Final-HF/'
#STATE_DICT = torch.load('/Users/cosmos/Documents/uncased_L-12_H-768_A-12/results/model_e10.tar', map_location=DEVICE)
TOKENIZER = BertTokenizerFast('/Users/cosmos/Documents/BERT/Final-HF/vocab.txt', lowercase=True)
#MODEL = BertForTokenClassification.from_pretrained(MODEL_PATH, state_dict=STATE_DICT['model_state_dict'], num_labels=12)


In [11]:
def get_label(offset, labels):
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'

In [12]:
tags_vals = ["UNKNOWN","Address","Links","University","Rewards and Achievements", "O", "Name", "Degree","Skills","College Name","Email Address","Designation","Companies worked at","Graduation Year","Years of Experience","Location"]
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i:t for i, t in enumerate(tags_vals)}

In [13]:
def process_resume(data, tokenizer, tag2idx, max_len, is_test=False):
    tok = tokenizer.encode_plus(data[0], max_length=max_len, return_offsets_mapping=True)
    curr_sent = {'orig_labels':[], 'labels': []}
    
    padding_length = max_len - len(tok['input_ids'])
    
    if not is_test:
        labels = data[1]['entities']
        labels.reverse()
        for off in tok['offset_mapping']:
            label = get_label(off, labels)
            curr_sent['orig_labels'].append(label)
            curr_sent['labels'].append(tag2idx[label])
        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)
    
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length)
    return curr_sent

In [14]:
class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2idx, max_len, is_test=False):
        self.resume = resume
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.tag2idx = tag2idx
        self.max_len = max_len
    
    def __len__(self):
        return len(self.resume)
    
    def __getitem__(self, idx):
        data = process_resume(self.resume[idx], self.tokenizer, self.tag2idx, self.max_len, self.is_test)
        return {
            'input_ids': torch.tensor(data['input_ids'], dtype=torch.long),
            'token_type_ids': torch.tensor(data['token_type_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(data['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(data['labels'], dtype=torch.long),
            'orig_label': data['orig_labels']
        }

In [15]:
total = len(data)
train_data, val_data = data[:180], data[180:]

In [16]:
train_d = ResumeDataset(train_data, TOKENIZER, tag2idx, MAX_LEN)
val_d = ResumeDataset(val_data, TOKENIZER, tag2idx, MAX_LEN)

In [17]:
train_sampler = RandomSampler(train_d)
train_dl = DataLoader(train_d, sampler=train_sampler, batch_size=8,collate_fn=lambda x: x )
#dataiter = iter(train_dl)
#data = dataiter.next()
val_dl = DataLoader(val_d, batch_size=8)

In [18]:
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters

In [19]:
def get_special_tokens(tokenizer, tag2idx):
    vocab = tokenizer.get_vocab()
    pad_tok = vocab["[PAD]"]
    sep_tok = vocab["[SEP]"]
    cls_tok = vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab

In [20]:
def annot_confusion_matrix(valid_tags, pred_tags):

    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t\t\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content

In [21]:
def flat_accuracy(valid_tags, pred_tags):
    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [22]:
MAX_GRAD_NORM = 1.0

In [23]:
import matplotlib.pyplot as plt

model = BertForTokenClassification.from_pretrained(MODEL_PATH, num_labels=len(tag2idx))
model.to(DEVICE);
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

Some weights of the model checkpoint at /Users/cosmos/Documents/BERT/Final-HF/ were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model chec

In [24]:
def train_and_save_model(
    model,
    tokenizer,
    optimizer,
    epochs,
    idx2tag,
    tag2idx,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader
):

    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)
    
    epoch = 0
    val_acc=[]
    ep=[]
    val_loss=[]
    for _ in trange(epochs, desc="Epoch"):
        epoch += 1

        # Training loop
        print("Starting training loop.")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):
            # Add batch to gpu
            
            #batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            tr_logits = tr_logits.cpu().detach().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.to("cpu").numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            )

            # Update parameters
            optimizer.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")
        
        
        """
        Validation loop
        """ 
        print("Starting validation loop.")

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:

            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            logits = logits.cpu().detach().numpy()
            label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            val_batch_preds = np.argmax(logits[preds_mask.squeeze()], axis=1)
            val_batch_labels = label_ids.to("cpu").numpy()
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)

            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        # Evaluate loss, acc, conf. matrix, and class. report on devset
        pred_tags = [idx2tag[i] for i in predictions]
        valid_tags = [idx2tag[i] for i in true_labels]
        cl_report = classification_report([valid_tags], [pred_tags])
        conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps

        # Report metrics
        print(f"Validation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")
        print(f"Confusion Matrix:\n {conf_mat}")
        val_acc.append(eval_accuracy)
        ep.append(epoch)
        val_loss.append(eval_loss)
    
    plt.plot(ep, val_acc, 'g', label='Validation accuracy')
    #plt.plot(ep, val_loss, 'b', label='Validation loss')
    plt.title('Validation acuuracy ')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()    

In [25]:
train_and_save_model(
    model, 
    TOKENIZER, 
    optimizer, 
    EPOCHS, 
    idx2tag, 
    tag2idx, 
    MAX_GRAD_NORM, 
    DEVICE, 
    train_dl, 
    val_dl
)

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]


Starting training loop.


KeyError: 21

In [ ]:
PyTorch

NameError: name 'PyTorch' is not defined

In [ ]:
def process_resume(text, tokenizer, max_len):
    tok = tokenizer.encode_plus(text, max_length=max_len, return_offsets_mapping=True)
    
    curr_sent = dict()
    
    padding_length = max_len - len(tok['input_ids'])
        
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + ([0] * padding_length)
    
    final_data = {
        'input_ids': torch.tensor(curr_sent['input_ids'], dtype=torch.long),
        'token_type_ids': torch.tensor(curr_sent['token_type_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(curr_sent['attention_mask'], dtype=torch.long),
        'offset_mapping': tok['offset_mapping']
    }
    
    return final_data

In [ ]:
tags_vals = ["UNKNOWN", "O", "Name", "Degree","Skills","College Name","Email Address","Designation","Companies worked at","Graduation Year","Years of Experience","Location"]
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i:t for i, t in enumerate(tags_vals)}

In [ ]:
def predict(model, tokenizer, idx2tag, tag2idx, device, test_resume):
    model.eval()
    data = process_resume(test_resume, tokenizer, MAX_LEN)
    input_ids, input_mask = data['input_ids'].unsqueeze(0), data['attention_mask'].unsqueeze(0)
    labels = torch.tensor([1] * input_ids.size(0), dtype=torch.long).unsqueeze(0)
    with torch.no_grad():
        outputs = model(
            input_ids,
            token_type_ids=None,
            attention_mask=input_mask,
            labels=labels,
        )
        tmp_eval_loss, logits = outputs[:2]
    
    logits = logits.cpu().detach().numpy()
    label_ids = np.argmax(logits, axis=2)
    
    entities = []
    for label_id, offset in zip(label_ids[0], data['offset_mapping']):
        curr_id = idx2tag[label_id]
        curr_start = offset[0]
        curr_end = offset[1]
        if curr_id != 'O':
            if len(entities) > 0 and entities[-1]['entity'] == curr_id and curr_start - entities[-1]['end'] in [0, 1]:
                entities[-1]['end'] = curr_end
            else:
                entities.append({'entity': curr_id, 'start': curr_start, 'end':curr_end})
    for ent in entities:
        ent['text'] = test_resume[ent['start']:ent['end']]
    return entities

In [ ]:
resume = data[41][0]
entities = predict(model, TOKENIZER, idx2tag, tag2idx, DEVICE, resume)